In [ ]:
!pip install pandas

In [ ]:
!pip install openpyxl

In [ ]:
!pip install tabulate

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from tabulate import tabulate

In [ ]:
#reading sample compute xls
df = pd.read_excel('sample_compute.xlsx', sheet_name=1, header=1)

In [ ]:
df.info()

In [ ]:
print(df.head())

In [ ]:
selected_columns = ["Order Ref", "Passenger Count", "Bid Value","Normalized Bid Price"]
rename_dict = {"Order Ref": "order_id", "Passenger Count": "pax_count", "Bid Value": "bid_value","Normalized Bid Price":"norm_bid_price"}

In [ ]:
df_selected = df[selected_columns].rename(columns=rename_dict)

In [ ]:
df_selected['total_bid_value'] = df_selected['pax_count'] * df_selected['bid_value']

In [ ]:
df_selected['total_norm_bid_value'] = df_selected['pax_count'] * df_selected['norm_bid_price']

In [ ]:
df_selected.info()

In [ ]:
print(tabulate(df_selected, headers='keys', tablefmt='psql'))

In [ ]:
stock_inventory = 4 #bag_size

# Core knapsack  implementation 

In [ ]:
def knapsack_from_dataframe(df, max_capacity):
     n = len(df)
     values = df["total_norm_bid_value"].tolist()
     weights = df["pax_count"].tolist()
     items = df["order_id"].tolist()
     dp = np.zeros((n + 1, max_capacity + 1))
     print(type(dp))
     print(dp)

    # Fill DP table
     for i in range(1, n + 1):
        for w in range(1, max_capacity + 1):
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]
     print(dp)
    
    # Traceback to find selected items
     w = max_capacity
     selected_items = []
     for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:  # Item was included
            selected_items.append(items[i - 1])
            w -= weights[i - 1]

     df_selected = df[df["order_id"].isin(selected_items)].reset_index(drop=True)

     return df_selected, dp[n][max_capacity]  # Return selected items and max value
             
    

#notes for self 
? what is dp table 
creates a Dynamic Programming (DP) table for solving the 0/1 Knapsack problem using bottom-up dynamic programming.
Breaking It Down
n + 1: Rows → Represents items (0 to n)
max_capacity + 1: Columns → Represents weight limits (0 to max_capacity)
np.zeros(...): Creates a 2D array filled with zeros to store intermediate results

Why Use a DP Table?
The DP table helps us store and reuse previously computed values, avoiding redundant calculations.
Each cell dp[i][w] stores the maximum value possible using the first i items within weight w.

    How Does It Get Filled?
dp[i][w] is updated iteratively based on:
Including the item (value[i] + dp[i-1][w-weight[i]])
Excluding the item (dp[i-1][w])
Taking the maximum of both

Example Update Process
Let's assume:

We have items: [(Value=60, Weight=10), (Value=100, Weight=20)]
max_capacity = 30
The DP table gets filled row by row, considering each item one by one.
After filling, the last cell (dp[n][max_capacity]) contains the maximum value that can be obtained.


In [ ]:
df_sack = knapsack_from_dataframe(df_selected,stock_inventory)

In [ ]:
type(df_sack)

In [ ]:
print(df_sack)

In [ ]:
for item in df_sack:
    print (item)

#  CORE ORCHESTRATION 

In [ ]:
sample_request_dict = {'bids': [{'bid_order_id': 'BID001', 'fare_basis_code': 'FLEX', 'purchase_history': '0', 'bid_value': '1121', 'pax_fft': 'GOLD', 'pax_count': '2'}, {'bid_order_id': 'BID002', 'fare_basis_code': 'CHOICE', 'purchase_history': '2', 'bid_value': '1305', 'pax_fft': 'SILVER', 'pax_count': '3'}, {'bid_order_id': 'BID003', 'fare_basis_code': 'SAVER', 'purchase_history': '3', 'bid_value': '1333', 'pax_fft': 'GOLD', 'pax_count': '1'}, {'bid_order_id': 'BID004', 'fare_basis_code': 'FLEX', 'purchase_history': '1', 'bid_value': '1183', 'pax_fft': 'NO FFT', 'pax_count': '2'}, {'bid_order_id': 'BID005', 'fare_basis_code': 'SAVER', 'purchase_history': '5', 'bid_value': '1120', 'pax_fft': 'SILVER', 'pax_count': '1'}, {'bid_order_id': 'BID006', 'fare_basis_code': 'FLEX', 'purchase_history': '0', 'bid_value': '1460', 'pax_fft': 'NO FFT', 'pax_count': '1'}, {'bid_order_id': 'BID007', 'fare_basis_code': 'CHOICE', 'purchase_history': '0', 'bid_value': '1084', 'pax_fft': 'NO FFT', 'pax_count': '1'}], 'stock_limit': '4'}
pdf = pd.DataFrame(sample_request_dict['bids'])

In [ ]:
pdf["purchase_history"] = pdf["purchase_history"].astype(int)
pdf["bid_value"] = pdf["bid_value"].astype(float)
pdf["pax_count"] = pdf["pax_count"].astype(int)


In [ ]:
print(pdf.columns)
pdf.info()

In [ ]:
print(tabulate(pdf, headers='keys', tablefmt='psql'))

In [ ]:
fft_score_df = pd.DataFrame({
    "pax_fft": ["GOLD", "SILVER", "NO FFT"],
    "fft_score": [1.2, 1.1, 1.0]
})

fbc_score_df = pd.DataFrame({
    "fare_basis_code": ["FLEX", "CHOICE", "SAVER"],
    "fbc_score": [1.1, 1.05, 1.0]
})

past_purchase_score = pd.DataFrame({
    'min': [0, 1, 4],
    'max': [0, 3, 9999],
    'score': [1, 0.8, 0.5]
})

In [ ]:
def assign_pph_score(val):
    row = past_purchase_score[(past_purchase_score['min'] <= val) & (past_purchase_score['max'] >= val)]
    return row['score'].values[0] if not row.empty else None  # Returns None if no match


In [ ]:
# joing with fft_score
pdf = pdf.merge(fft_score_df, on="pax_fft", how="left")

# joing with fbc_score
pdf = pdf.merge(fbc_score_df, on="fare_basis_code", how="left")


In [ ]:
# using function to assign 
pdf['ph_score'] = pdf['purchase_history'].apply(assign_pph_score)


In [ ]:
# creation of normalized_bid_val
pdf['bid_normalized_score'] = np.round(pdf[['fft_score', 'fbc_score', 'ph_score', 'bid_value']].prod(axis=1), 0).astype(int)

In [ ]:
pdf['total_normalized_bid'] = pdf['pax_count'] * pdf['bid_normalized_score']

In [201]:
pdf['total_bid_amount'] = pdf['pax_count'] * pdf['bid_value']

In [ ]:
pdf['bid_priority_score'] = pdf['total_normalized_bid'].rank(ascending=False, method='dense').astype(int)

In [ ]:
selected_columns = ['bid_order_id', 'total_normalized_bid', 'bid_priority_score','pax_count']

print(tabulate(pdf[selected_columns], headers='keys', tablefmt='pretty'))


In [222]:
def ksack_new(df, max_capacity):
     n = len(df)
     values = df["total_normalized_bid"].tolist()
     weights = df["pax_count"].tolist()
     items = df["bid_order_id"].tolist()
     dp = np.zeros((n + 1, max_capacity + 1))
     print(type(dp))
     print(dp)

    # Fill DP table
     for i in range(1, n + 1):
        for w in range(1, max_capacity + 1):
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]
     print(dp)
    
    # Traceback to find selected items
     w = max_capacity
     selected_items = []
     for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:  # Item was included
            selected_items.append(items[i - 1])
            w -= weights[i - 1]

     df_selected = df[df["bid_order_id"].isin(selected_items)].reset_index(drop=True)

     return df_selected, dp[n][max_capacity]  # Return selected items and max value
           

In [223]:
ksak_val = ksack_new(pdf,4)

<class 'numpy.ndarray'>
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[   0.    0.    0.    0.    0.]
 [   0.    0. 2960. 2960. 2960.]
 [   0.    0. 2960. 3618. 3618.]
 [   0. 1280. 2960. 4240. 4898.]
 [   0. 1280. 2960. 4240. 5042.]
 [   0. 1280. 2960. 4240. 5042.]
 [   0. 1606. 2960. 4566. 5846.]
 [   0. 1606. 2960. 4566. 5846.]]


In [224]:
ksak_val[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bid_order_id          3 non-null      object 
 1   fare_basis_code       3 non-null      object 
 2   purchase_history      3 non-null      int64  
 3   bid_value             3 non-null      float64
 4   pax_fft               3 non-null      object 
 5   pax_count             3 non-null      int64  
 6   fft_score             3 non-null      float64
 7   fbc_score             3 non-null      float64
 8   ph_score              3 non-null      float64
 9   bid_normalized_score  3 non-null      int64  
 10  total_normalized_bid  3 non-null      int64  
 11  bid_priority_score    3 non-null      int64  
 12  total_bid_amount      3 non-null      float64
dtypes: float64(5), int64(5), object(3)
memory usage: 440.0+ bytes


In [202]:
pdf

,bid_order_id,fare_basis_code,purchase_history,bid_value,pax_fft,pax_count,fft_score,fbc_score,ph_score,bid_normalized_score,total_normalized_bid,bid_priority_score,total_bid_amount
0,BID001,FLEX,0,1121.0,GOLD,2,1.2,1.10,1.0,1480,2960,2,2242.0
1,BID002,CHOICE,2,1305.0,SILVER,3,1.1,1.05,0.8,1206,3618,1,3915.0
2,BID003,SAVER,3,1333.0,GOLD,1,1.2,1.00,0.8,1280,1280,5,1333.0
3,BID004,FLEX,1,1183.0,NO FFT,2,1.0,1.10,0.8,1041,2082,3,2366.0
4,BID005,SAVER,5,1120.0,SILVER,1,1.1,1.00,0.5,616,616,7,1120.0
5,BID006,FLEX,0,1460.0,NO FFT,1,1.0,1.10,1.0,1606,1606,4,1460.0
6,BID007,CHOICE,0,1084.0,NO FFT,1,1.0,1.05,1.0,1138,1138,6,1084.0


In [211]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bid_order_id          7 non-null      object 
 1   fare_basis_code       7 non-null      object 
 2   purchase_history      7 non-null      int64  
 3   bid_value             7 non-null      float64
 4   pax_fft               7 non-null      object 
 5   pax_count             7 non-null      int64  
 6   fft_score             7 non-null      float64
 7   fbc_score             7 non-null      float64
 8   ph_score              7 non-null      float64
 9   bid_normalized_score  7 non-null      int64  
 10  total_normalized_bid  7 non-null      int64  
 11  bid_priority_score    7 non-null      int64  
 12  total_bid_amount      7 non-null      float64
dtypes: float64(5), int64(5), object(3)
memory usage: 856.0+ bytes


In [214]:
print (pdf.columns)

Index(['bid_order_id', 'fare_basis_code', 'purchase_history', 'bid_value',
       'pax_fft', 'pax_count', 'fft_score', 'fbc_score', 'ph_score',
       'bid_normalized_score', 'total_normalized_bid', 'bid_priority_score',
       'total_bid_amount'],
      dtype='object')


# Greedy fetch algorithm

In [207]:
df_greed = pdf.sort_values(by="bid_priority_score", ascending=True)

In [208]:
capacity = 4
total_weight = 0
total_cost = 0
selected_items = []

In [209]:
for _, row in df_greed.iterrows():
    weight = row['pax_count']  # Extract as scalar
    if total_weight + weight <= capacity:
        selected_items.append(row['bid_order_id'])
        total_weight += row['pax_count']
        total_cost += row['total_bid_amount']
    if total_weight == capacity:
        break;

In [210]:
total_cost

5375.0

In [231]:
selected_items

['BID002', 'BID006']

In [232]:
g_result = df_greed[df_greed['bid_order_id'].isin(selected_items)]

In [233]:
g_result

,bid_order_id,fare_basis_code,purchase_history,bid_value,pax_fft,pax_count,fft_score,fbc_score,ph_score,bid_normalized_score,total_normalized_bid,bid_priority_score,total_bid_amount
1,BID002,CHOICE,2,1305.0,SILVER,3,1.1,1.05,0.8,1206,3618,1,3915.0
5,BID006,FLEX,0,1460.0,NO FFT,1,1.0,1.10,1.0,1606,1606,4,1460.0


# response making

In [216]:
normalized_score = pdf.rename(columns={
    'bid_order_id': 'bid_id',
    'total_bid_amount': 'total_bid_vale',
    'fft_score': 'fft_score',
    'fbc_score': 'fbc_score',
    'ph_score': 'purchase_history_score',
    'bid_normalized_score': 'normalized_bid',
    'total_normalized_bid' : 'total_normalized_bid',
    'bid_priority_score' : 'bid_priority_score'
})[['bid_id', 'total_bid_vale', 'fft_score', 'fbc_score', 'purchase_history_score', 'normalized_bid','total_normalized_bid','bid_priority_score']]


In [217]:
json_response = {"normalized_score": normalized_score.to_dict(orient='records')}

In [ ]:
json_response

In [225]:
ks_df = ksak_val[0]

In [226]:
print(ks_df.columns)

Index(['bid_order_id', 'fare_basis_code', 'purchase_history', 'bid_value',
       'pax_fft', 'pax_count', 'fft_score', 'fbc_score', 'ph_score',
       'bid_normalized_score', 'total_normalized_bid', 'bid_priority_score',
       'total_bid_amount'],
      dtype='object')


In [228]:
ks_result = ks_df.rename(
    columns={
        'bid_order_id': 'bid_id',
        'bid_priority_score' : 'bid_priority_score',
        'pax_count' : 'pax_count',
        'total_normalized_bid': 'total_normalized_bid',
        'total_bid_amount' : 'total_bid_amount'
        })[['bid_id','bid_priority_score','pax_count','total_normalized_bid','total_bid_amount']]


In [229]:
json_response = {"ks_result": ks_result.to_dict(orient='records')}

In [230]:
json_response

{'ks_result': [{'bid_id': 'BID001',
   'bid_priority_score': 2,
   'pax_count': 2,
   'total_normalized_bid': 2960,
   'total_bid_amount': 2242.0},
  {'bid_id': 'BID003',
   'bid_priority_score': 5,
   'pax_count': 1,
   'total_normalized_bid': 1280,
   'total_bid_amount': 1333.0},
  {'bid_id': 'BID006',
   'bid_priority_score': 4,
   'pax_count': 1,
   'total_normalized_bid': 1606,
   'total_bid_amount': 1460.0}]}

In [ ]:
g_result.rename()